In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader,Dataset
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from collections import defaultdict
from sklearn.metrics import f1_score,accuracy_score
from torchvision import models

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    # Ensure images are converted to tensors
])

In [ ]:
dataset = datasets.ImageFolder(root=r"C:\Users\mushfika_rahman1\Documents\Research\research-summer-2024\LIMUC\train_and_validation_sets",transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
for batch_images, batch_labels in dataloader:
    # Print the type of batch_images and batch_labels
    print(f"Type of batch_images: {type(batch_images)}")
    print(f"Type of batch_labels: {type(batch_labels)}")

    # Print the shape of batch_images to get the batch size and image dimensions
    print(f"Batch image tensor size: {batch_images.size()}")  # Should print [batch_size, channels, height, width]
    print(f"Batch labels: {batch_labels}")

    # Optionally, print the batch size only
    batch_size = batch_images.size(0)
    print(f"Batch size: {batch_size}")
    
    # Optionally, you can also check if the elements in the batch are tensors
    if isinstance(batch_images, torch.Tensor):
        print("Batch images are tensors.")
    else:
        print("Batch images are not tensors.")
    
    # You only need to check the first batch to confirm
    break

In [4]:
def rotate_image(x):
    angles = [0, 90, 180, 270]
    rotated_images=[]
    rotated_labels=[]
    for angle in angles:
        rotated_img = TF.rotate(x,angle)
        rotated_images.append(rotated_img)
        rotated_labels.append(angles.index(angle))
        #rotated_labels.append(angle)
    return rotated_images,rotated_labels

In [5]:
class CustomRotatedDataset(Dataset):
    def __init__(self, original_dataset):
        self.dataset = original_dataset

    def __len__(self):
        return len(self.dataset)*4

    def __getitem__(self, index):
        original_index = index // 4 
        angle_index = index % 4 
        
        img, _ = self.dataset[original_index]
        rotated_img, rotated_labels = rotate_image(img)
        return rotated_img[angle_index], rotated_labels[angle_index]

In [6]:
subset_dataset = datasets.ImageFolder(root=r"C:\Users\mushfika_rahman1\Documents\Research\research-summer-2024\LIMUC\subset",transform=transform)
subset_loader = DataLoader(subset_dataset, batch_size=16, shuffle=True)

In [7]:
rotated_dataset = CustomRotatedDataset(subset_dataset)
rotated_loader = DataLoader(rotated_dataset, batch_size=16,shuffle=True)

In [8]:
model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 classes for 0°, 90°, 180°, 270°

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
save_dir = 'resnet18_model_weights_LIMUC'
os.makedirs(save_dir, exist_ok=True)

In [11]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Set the model to training mode
    
    for images, labels in rotated_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(rotated_loader):.4f}')
    if (epoch+1)%5==0:
        save_path = os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth')
        torch.save(model.state_dict(), save_path)
        print(f'Model weights saved for epoch {epoch + 1} at {save_path}')


Epoch [1/10], Loss: 0.3692
Epoch [2/10], Loss: 0.0500
Epoch [3/10], Loss: 0.0089
Epoch [4/10], Loss: 0.0031
Epoch [5/10], Loss: 0.0016
Model weights saved for epoch 5 at resnet18_model_weights_LIMUC\model_epoch_5.pth
Epoch [6/10], Loss: 0.0103
Epoch [7/10], Loss: 0.0237
Epoch [8/10], Loss: 0.0139
Epoch [9/10], Loss: 0.1017
Epoch [10/10], Loss: 0.0229
Model weights saved for epoch 10 at resnet18_model_weights_LIMUC\model_epoch_10.pth


In [13]:
model.fc = nn.Linear(num_ftrs, len(subset_dataset.classes))
model.load_state_dict(torch.load('resnet18_model_weights_LIMUC/model_epoch_5.pth'))  # Replace with the correct path
model = model.to(device)

In [15]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Set the model to training mode
    
    for images, labels in subset_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(subset_loader):.4f}')

# Save the fine-tuned model
torch.save(model.state_dict(), 'resnet18_finetuned_subset.pth')
print("Model fine-tuned on subset dataset and weights saved.")

Epoch [1/10], Loss: 1.7125
Epoch [2/10], Loss: 1.3512
Epoch [3/10], Loss: 1.0795
Epoch [4/10], Loss: 0.7704
Epoch [5/10], Loss: 0.5458
Epoch [6/10], Loss: 0.4477
Epoch [7/10], Loss: 0.2926
Epoch [8/10], Loss: 0.1432
Epoch [9/10], Loss: 0.0928
Epoch [10/10], Loss: 0.1826
Model fine-tuned on subset dataset and weights saved.


In [16]:
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in subset_loader:  
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy:.4f}')

# Calculate F1 Score
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9250
F1 Score: 0.9261


In [17]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_dataset = datasets.ImageFolder(root=r"C:\Users\mushfika_rahman1\Documents\Research\research-summer-2024\LIMUC\test_set", transform=test_transform)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [19]:
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:  
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy on test set: {accuracy:.4f}')

# Calculate F1 Score
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'F1 Score on test set: {f1:.4f}')

Accuracy on test set: 0.2888
F1 Score on test set: 0.2828
